# Week 11: Unsupervised Learning Assignment

## Principal Component Analysis (PCA)
In this section, you will work with the PCA algorithm in order to understand its definition and explore its uses.

### Principle of Maximum Variance: what is PCA supposed to do?
First of all, let us recall the principle/assumption of PCA:
1. What is the variance?
3. What is the covariance?
3. How do we compute covariance?
2. What is the meaning of the principle of maximum variance?
4. Why do we need this principle?
5. Does the principle always apply?

**Answers:** enter your answers here.

## Implementation: how is PCA implemented?
Here we implement the basic steps of PCA and assemble them.

### Importing libraries
We start by importing the *numpy* library for performing matrix computations, the *pyplot* library for plotting data, and the *syntheticdata* module to import synthetic data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import syntheticdata

### Centering the Data
Implement a function with the following signature to center the data as explained in *Marsland*.

In [ ]:
def center_data(A):
    # INPUT:
    # A    [NxM] numpy data matrix (N samples, M features)
    #
    # OUTPUT:
    # X    [NxM] numpy centered data matrix (N samples, M features)
    
    return None

Test your function, checking the following assertion on *testcase*:

In [ ]:
testcase = np.array([[3.,11.,4.3],[4.,5.,4.3],[5.,17.,4.5]])
answer = np.array([[-1.,0.,4.3-(13.1/3.)],[0,-6.,4.3-(13.1/3.)],[1,6.,4.5-(13.1/3.)]])
assert(np.all(center_data(testcase)==answer))

### Computing Covariance Matrix
Implement a function with the following signature to compute the covariance matrix as explained in *Marsland*.

In [ ]:
def compute_covariance_matrix(A):
    # INPUT:
    # A    [NxM] numpy data matrix (N samples, M features)
    #
    # OUTPUT:
    # C    [NxM] numpy covariance matrix (N samples, M features)
    
    return None

Test your function, checking the following assertion on *testcase*:

In [ ]:
testcase = np.array([[22.,11.,5.5],[10.,5.,2.5],[34.,17.,8.5]])
answer = np.array([[580.,290.,145.],[290.,145.,72.5],[145.,72.5,36.25]])
assert(np.all(compute_covariance_matrix(testcase)==answer))

### Computing eigenvalues and eigenvectors
Use the linear algebra package of *numpy* and its function *linalg.eig()* to compute eigenvalues and eigenvectors.

In [ ]:
def compute_eigenvalue_eigenvectors(A):
    # INPUT:
    # A    [DxD] numpy matrix
    #
    # OUTPUT:
    # eigval    [D] numpy vector of eigenvalues
    # eigvec    [DxD] numpy array of eigenvectors
    
    return None

Test your function, checking the following assertion on *testcase*:

In [ ]:
testcase = np.array([[2,0,0],[0,5,0],[0,0,3]])
answer1 = np.array([2.,5.,3.])
answer2 = np.array([[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]])
x,y = compute_eigenvalue_eigenvectors(testcase)
assert(np.all(x==answer1))
assert(np.all(y==answer2))

### Sorting eigenvalues and eigenvectors
Implement a function with the following signature to sort eigenvalues and eigenvectors as explained in *Marsland*.

Remember that eigenvalue *eigval[i]* corresponds to eigenvector *eigvec[:,i]*.

In [ ]:
def sort_eigenvalue_eigenvectors(eigval, eigvec):
    # INPUT:
    # eigval    [D] numpy vector of eigenvalues
    # eigvec    [DxD] numpy array of eigenvectors
    #
    # OUTPUT:
    # sorted_eigval    [D] numpy vector of eigenvalues
    # sorted_eigvec    [DxD] numpy array of eigenvectors
    
    return None

Test your function, checking the following assertion on *testcase*:

In [ ]:
testcase = np.array([[2,0,0],[0,5,0],[0,0,3]])
answer1 = np.array([5.,3.,2.])
answer2 = np.array([[0.,0.,1.],[1.,0.,0.],[0.,1.,0.]])
x,y = compute_eigenvalue_eigenvectors(testcase)
x,y = sort_eigenvalue_eigenvectors(x,y)
assert(np.all(x==answer1))
assert(np.all(y==answer2))

### PCA Algorithm
Implement a function with the following signature to compute PCA as explained in *Marsland*, using the functions implemented above.

In [ ]:
def PCA(A,m):
    # INPUT:
    # A    [NxM] numpy data matrix (N samples, M features)
    # m    integer number denoting the number of learned features (m <= M)
    #
    # OUTPUT:
    # pca_eigvec    [mxM] numpy matrix containing the eigenvectors (m eigenvectors, M dimensions)
    # P    [Nxm] numpy PCA data matrix (N samples, m features)
    
    return None

Test your function, checking the following assertion on *testcase*:

In [ ]:
testcase = np.array([[22.,11.,5.5],[10.,5.,2.5],[34.,17.,8.5]])
x,y = PCA(testcase,2)
import pickle
answer1_file = open('PCAanswer1.pkl','rb'); answer2_file = open('PCAanswer2.pkl','rb')
answer1 = pickle.load(answer1_file); answer2 = pickle.load(answer2_file)
assert(np.all(x==answer1))
assert(np.all(y==answer2))

## Understanding: how does PCA work?
We now use the PCA algorithm you implemented on a toy data set in order to understand its inner workings.

### Loading the data
The module *syntheticdata* provides a small synthetic dataset of dimension [100x2] (100 samples, 2 features).

In [ ]:
X = syntheticdata.get_synthetic_data1()

### Visualizing the data
Visualize the synthetic data using the function *scatter()* from the *matplotlib* library.

In [ ]:
plt.scatter(X[:,0],X[:,1])

### Visualize the centered data
Notice that the data visualized above is not centered on the origin (0,0). Use the function defined above to center the data, and then replot it.

In [ ]:
None

### Visualize the first eigenvector
Visualize the vector defined by the first eigenvector.
To do this you need to:
- Use the *PCA()* function to recover the eigenvectors
- Plot the centered data as done above 
- The first eigenvector is a 2D vector (x0,y0). This defines a vector with origin in (0,0) and head in (x0,y0). Use the function *plot()* from matplotlib to plot a line over the first eigenvector.

In [ ]:
pca_eigvec, _ = None
first_eigvec = None

plt.scatter(None,None)

x = np.linspace(-5, 5, 1000)
y = first_eigvec[1]/first_eigvec[0] * x
plt.plot(x,y)

### Visualize the PCA projection
Finally, use the *PCA()* algorithm to project on a single dimension and visualize the result using again the *scatter()* function.

In [ ]:
_,P = None
plt.scatter(None,np.ones(X.shape[0]))

## Evaluation: when are the results of PCA sensible?
So far we have used PCA on synthetic data. Let us now imagine we are using PCA as a pre-processing step before a classification task. This is a common setup with high-dimensional data. We explore when the use of PCA is sensible.

### Loading the first set of labels
The function *get_synthetic_data_with_labels1()* from the module *syntethicdata* provides a first labeled dataset.

In [ ]:
X,y = syntheticdata.get_synthetic_data_with_labels1()

### Running PCA
Process the data using the PCA algorithm and project it in one dimension. Plot the labeled data using *scatter()* before and after running PCA. Comment on the results.

In [ ]:
plt.scatter(None,None,c=y[:,0])

plt.figure()
_,P = None
plt.scatter(None,np.ones(X.shape[0]),c=y[:,0])

**Comment:** enter your comment here.

### Loading the second set of labels
The function *get_synthetic_data_with_labels2()* from the module *syntethicdata* provides a second labeled dataset.

In [ ]:
X,y = syntheticdata.get_synthetic_data_with_labels2()

### Running PCA
As before, process the data using the PCA algorithm and project it in one dimension. Plot the labeled data using *scatter()* before and after running PCA. Comment on the results.

In [ ]:
None

**Comment:** enter your comment here.

How would the result change if you were to consider the second eigenvector? Or if you were to consider both eigenvectors?

**Answer:** enter your answer here.

## Case study 1: PCA for visualization
We now consider the *iris* dataset, a simple collection of data (N=150) describing iris flowers with four (M=4) features. The features are: Sepal Length, Sepal Width, Petal Length and Petal Width. Each sample has a label, identifying each flower as one of 3 possible types of iris: Setosa, Versicolour, and Virginica.

Visualizing a 4-dimensional dataset is impossible; therefore we will use PCA to project our data in 2 dimensions and visualize it.

### Loading the data
The function *get_iris_data()* from the module *syntethicdata* returns the *iris* dataset. It returns a data matrix of dimension [150x4] and a label vector of dimension [150].

In [ ]:
X,y = syntheticdata.get_iris_data()

### Visualizing the data by selecting features
Try to visualize the data (using label information) by randomly selecting two out of the four features of the data. You may try different pairs of features.

In [ ]:
None

### Visualizing the data by PCA
Process the data using PCA and visualize it (using label information). Compare with the previous visualization and comment on the results.

In [ ]:
None

**Comment:** enter your comment here.

## Case study 2: PCA for compression
We now consider the *faces in the wild (lfw)* dataset, a collection of pictures (N=1280) of people. Each pixel in the image is a feature (M=2914).

### Loading the data
The function *get_lfw_data()* from the module *syntethicdata* returns the *lfw* dataset. It returns a data matrix of dimension [1280x2914] and a label vector of dimension [1280]. It also returns two parameters, $h$ and $w$, reporting the height and the width of the images (these parameters are necessary to plot the data samples as images).

In [ ]:
X,y,h,w = syntheticdata.get_lfw_data()

### Inspecting the data
Choose one datapoint to visualize (first coordinate of the matrix $X$) and use the function [imshow()](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.imshow.html) to plot and inspect some of the pictures.

Notice that *imshow* receives as a first argument an image to be plot; the image must be provided as a rectangular matrix, therefore we reshape a sample from the matrix $X$ to have height $h$ and width $w$. The parameter *cmap* specifies the color coding; in our case we will visualize the image in black-and-white with different gradations of grey.

In [ ]:
plt.imshow(X[0,:].reshape((h, w)), cmap=plt.cm.gray)

### Implementing a compression-decompression function
Implement a function that first uses PCA to project samples in low-dimensions, and then reconstruct the original image.

*Hint:* Most of the code is the same as the previous PCA() function you implemented. You may want to refer to *Marsland* to check out how reconstruction is performed.

In [ ]:
def encode_decode_PCA(A,m):
    # INPUT:
    # A    [NxM] numpy data matrix (N samples, M features)
    # m    integer number denoting the number of learned features (m <= M)
    #
    # OUTPUT:
    # Ahat [NxM] numpy PCA reconstructed data matrix (N samples, M features)
    
    return None

### Compressing and decompressing the data
Use the implemented function to encode and decode the data by projecting on a lower dimensional space of dimension 200 (m=200).

In [ ]:
Xhat = encode_decode_PCA(X,None)

### Inspecting the reconstructed data
Use the function *imshow* (as done before) to visualize and compare original and reconstructed pictures. Comment on the results.

In [ ]:
None

**Comment:** enter your comment here.

### Evaluating different compressions
Use the previous setup to generate compressed images using different values of dimensions in the PCA algorithm (e.g.: 100, 200, 500, 1000). Plot and comment on the results.

In [ ]:
plt.imshow(X[0,:].reshape((h, w)), cmap=plt.cm.gray)

plt.figure()
Xhat = encode_decode_PCA(X,None)
plt.imshow(Xhat[0,:].reshape((h, w)), cmap=plt.cm.gray)

None

**Comment:** enter your comment here.

## K-Means Clustering
In this section you will use the *k-means clustering* algorithm to perform unsupervised clustering.

### Importing scikit-learn library
We start by importing the module *cluster.KMeans* from the standard machine learning library *scikit-learn*.

In [ ]:
from sklearn.cluster import KMeans

### Loading the data
We will use once again the *iris* data set. The function *get_iris_data()* from the module *syntethicdata* returns the *iris* dataset. It returns a data matrix of dimension [150x4] and a label vector of dimension [150].

In [ ]:
X,y = syntheticdata.get_iris_data()

### Projecting the data using PCA
To allow for visualization, we project our data in two dimensions as we did previously. This step is not necessary, and we may want to try to use *k-means* later without the PCA pre-processing. However, we use PCA, as this will allow for an easy visualization.

In [ ]:
_,P = None

### Running k-means
Use the class *KMeans* to fit and predict the output of the *k-means* algorithm on the projected data. Run the algorithm using the following values of $k=\{2,3,4,5\}$. 

In [ ]:
KM = KMeans(None)
yhat2 = KM.fit_predict(P)

None

### Qualitative assessment
Plot the results of running the k-means algorithm, compare with the true labels, and comment.

In [ ]:
plt.scatter(P[:,0],P[:,1],c=None)
plt.title('k=2')

None

plt.figure()
plt.scatter(P[:,0],P[:,1],c=y)
plt.title('Original data')

**Comment:** enter your comment here.

## Optional 1: PCA Tuning
If we use PCA for compression or decompression, it may not be trivial to decide how many dimensions to keep. In this section we review a principled way to decide how many dimensions to keep.

The number of dimensions to keep is the only *hyper-parameter* of PCA. A method designed to decide how many dimensions/eigenvectors is the *proportion of variance*:
$$ \textrm{POV}=\frac{\sum_{i=1}^{m}{\lambda_{i}}}{\sum_{j=1}^{M}{\lambda_{j}}}, $$
where $\lambda$ are eigenvalues, $M$ is the dimensionality of the original data, and $m$ is the chosen lower dimensionality. 

Using the $POV$ formula we may select a number $m$ of dimensions/eigenvalues so that the proportion of variance is, for instance, equal to 95%.

Implement a new PCA for encoding and decoding that receives in input not the number of dimensions for projection, but the amount of proportion of variance to be preserved.

In [ ]:
def encode_decode_PCA_with_pov(A,p):
    # INPUT:
    # A    [NxM] numpy data matrix (N samples, M features)
    # p    float number between 0 and 1 denoting the POV to be preserved
    #
    # OUTPUT:
    # Ahat [NxM] numpy PCA reconstructed data matrix (N samples, M features)
    # m    integer reporting the number of dimensions selected
    
    return None

Import the *lfw* dataset using the *get_lfw_data()* in *syntheticdata*. Use the implemented function to encode and decode the data by projecting on a lower dimensional space such that POV=0.9. Use the function *imshow* to plot and compare original and reconstructed pictures. Comment on the results.

In [ ]:
X,y,h,w = syntheticdata.get_lfw_data()

In [ ]:
Xhat,m = encode_decode_PCA_with_pov(X,None)

In [ ]:
plt.imshow(X[0,:].reshape((h, w)), cmap=plt.cm.gray)
plt.figure()
plt.imshow(Xhat[0,:].reshape((h, w)), cmap=plt.cm.gray)

**Comment:** enter your comment here.

## Optional 2: Quantitative Assessment of K-Means

We used k-means for clustering and we assessed the results qualitatively by visualizing them. However we often want to be able to measure in a quantitative way how good the clustering was. To do this, we will use a classification task to evaluete numerically the goodness of the representation learned via k-means.

Reload the *iris* dataset. Import a standard *RidgeClassifier* from the module *sklearn.linear_model*. Use the k-means representations learned previously (*yhat2,...,yhat5*) and the true label to train the classifier. Evaluate your model on the training data (we do not have a test set, so this procedure will assess the model fit instead of generalization) using the *accuracy_score()* function from the *sklearn.metrics* module. Plot a graph showing how the accuracy score varies when changing the value of k. Comment on the results.

In [ ]:
X,y = syntheticdata.get_iris_data()

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
lr = RidgeClassifier()

lr.fit(None)
yhat = lr.predict(None)
print(metrics.accuracy_score(y,yhat))

lr.fit(yhat2.reshape([yhat2.size,1]),y)
yhat = lr.predict(yhat2.reshape([yhat2.size,1]))
acc2 = metrics.accuracy_score(y,yhat)

None

**Comment:** enter your comment here.

# Conclusions 

In this notebook we studied **unsupervised learning** considering two important and representative algorithms: **PCA** and **k-means**.

First, we implemented the PCA algorithm step by step; we then run the algorithm on synthetic data in order to see its inner workings and evaluate when it may make sense to use it and when not. We then considered two typical uses of PCA: for **visualization** on the *iris* dataset, and for **compression-decompression** on the *lfw* dataset.

We then moved to consider the k-means algorithm. In this case we used the implementation provided by *scikit-learn* and we applied it to another prototypical unsupervised learning problem: **clustering**; we used *k-means* to process the *iris* dataset and we evaluated the results visually.

In the final optional part, we considered two additional questions that may arise when using the above algorithms. For PCA, we considered the problem of **selection of hyper-parameters**, that is, how we can select the hyper-parameter of our algorithm in a reasonable fashion. For k-means, we considered the problem of the **quantitative evaluation** of our results, that is, how can we measure the performance or usefulness of our algorithms. 